In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
def shape_space_basis_func(x, n):
    """Shape space basis function. Shape space and class functions combined."""
    return [math.comb(n, i) * x**(i+0.5) * (1-x)**(n-i+1) for i in range(n+1)]


def shape_space_thickness(psi, n, A):
    """Theoretical surface generated using shape space parameters A."""
    return np.dot(A, shape_space_basis_func(psi, n))


def shape_space_coords(upper_coeffs, lower_coeffs, sample_points=10):
    """Generate aerofoil shape from shape space coefficients"""
    polynomial_order = len(upper_coeffs) - 1

    x = np.linspace(0, 1, sample_points)
    upper_coords = [shape_space_thickness(v, polynomial_order, upper_coeffs) for v in x]
    lower_coords = [shape_space_thickness(v, polynomial_order, lower_coeffs) for v in x]

    upper_coords = list(zip(x, upper_coords))  # type: ignore
    lower_coords = list(zip(x, lower_coords))  # type: ignore

    # trailing edge -> upper surface -> leading edge -> lower surface -> trailing edge
    coords = upper_coords[::-1] + lower_coords[1:]

    return coords, upper_coords, lower_coords  # type: ignore


def aerofoil_mesh(upper_surface_coeffs, lower_surface_coeffs, height=1):
    coords, upper_coords, lower_coords = shape_space_coords(upper_surface_coeffs, lower_surface_coeffs)

    span_vertices = []
    top_vertices = []

    # spanwise
    for i in range(len(coords) - 1):
        span_vertices.extend(
            [
                (*coords[i], height),  # 0
                (*coords[i], 0),  # 1
                (*coords[i + 1], height),  # 3
                (*coords[i], height),  # 0
                (*coords[i + 1], height),  # 3
                (*coords[i], 0),  # 1
                (*coords[i + 1], 0),  # 2
                (*coords[i + 1], height),  # 3
            ],
        )

    # top surface
    for i in range(len(upper_coords) - 1):
        top_vertices.extend(
            [
                (*upper_coords[i], height),  # 0
                (*lower_coords[i], height),  # 1
                (*upper_coords[i + 1], height),  # 3
                (*upper_coords[i], height),  # 0
                (*upper_coords[i + 1], height),  # 3
                (*lower_coords[i], height),  # 1
                (*lower_coords[i + 1], height),  # 2
                (*upper_coords[i + 1], height),  # 3
            ],
        )

    return span_vertices, top_vertices

In [ ]:
upper_surface_coeffs = [0.1679916, 0.15851636, 0.1313818, 0.15751629]
lower_surface_coeffs = [-0.1679916, -0.15851636, -0.1313818, -0.15751629]

span_vertices, top_vertices = aerofoil_mesh(upper_surface_coeffs, lower_surface_coeffs)

span_vertices = np.array(span_vertices)
top_vertices = np.array(top_vertices)

In [ ]:
plt.rcParams.update({'font.size': 22})

fig = plt.figure(figsize=(20,20))
ax  = fig.add_subplot(111, projection = '3d')

zipped_span = list(zip(span_vertices, np.roll(span_vertices,1, axis=0)))
zipped_top = list(zip(top_vertices, np.roll(top_vertices,1, axis=0)))

for i,j in zipped_span:
    ax.plot([i[0],j[0]], [i[1],j[1]], [i[2],j[2]],color = 'k')

for i,j in zipped_top[1:]:
    ax.plot([i[0],j[0]], [i[1],j[1]], [i[2],j[2]],color = 'k')

ax.scatter(coords[:,0], coords[:,1], coords[:,2], color = 'b', marker = "o")


# force equal axis
max_range = 1
Xb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][0].flatten() + 0.5*(1)
Yb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][1].flatten() + 0.5*(0.2)
Zb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][2].flatten() + 0.5*(1)
for xb, yb, zb in zip(Xb, Yb, Zb):
   ax.plot([xb], [yb], [zb], 'w')

plt.show()